In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Pretendo hacer posible el uso de diferentes metodos para solucionar
# las ecuaciones diferenciales descritas por dY/dt = der_fun
# Como no todos los metodos requieren la misma informacion sobre los Ys
# y los ts ya existentes (por ejemplo multistep ones like the Adams-Bash one
# requieren de varios momentos anteriores)

def runge_kutta_4th(fun_der, constantes, Ys, dYsdt, timestep, ts):
    # Ys es un diccionario con claves los elementos del array ts
    # dYsdt es un diccionario con claves los elementos del array ts #Asumo, creo que correctamente, que el tiempo me determina todo el estado
    # timestep es el time step que utiliza el metodo    
    
    #TODO validación de argumentos
    
    # Calculo de el valor de Y luego de un timestep
    t_act = ts[-1]
    Y_act = Ys[t_act]
    
    k1 = fun_der(Y_act, t_act, constantes, dYsdt)
    k2 = fun_der(Y_act + timestep * k1 / 2., t_act + timestep / 2., constantes, dYsdt)
    k3 = fun_der(Y_act + timestep * k2 / 2., t_act + timestep / 2., constantes, dYsdt)
    k4 = fun_der(Y_act + timestep * k3, t_act + timestep, constantes, dYsdt)
    
    # Que quien se encargue de guardarlo sea alguien mas, pues, puede que
    # decida simplemente calcular el Y en un tiempo que ya existe, solo para
    # utilizarlo en algo mas, quiza para no obligar que se use solo un metodo
    # para la solucion de los sistemas de masas
    return Y_act + timestep*(k1/6. + k2/3. + k3/3. + k4/6.)

In [ ]:
class Sistema_Masas:
    
    def __init__(self, masas, Y_0, fun_derivada, #Cambio a Y_0 para no forzar aun la convencion que usare de posiciones velocidades
                 t_max = 10., 
                 metodo = runge_kutta_4th, timestp = None, cifras_fpos = 9):
        
        #TODO checks
        # check masas positivas, vectores 3D y mismo tamano y misma cantidad que de masas, 
        # t_max positivo, timestp > 0,
        # actual_time > 0 y < a t_max redondeado + timestp, 
        # err_rel_fpos > 0
        
        # Guardar variables necesarias para enteder el sistema y la informacion
        ## guardada
        self.masas = np.array(masas) #any sequence type probably
        self.N = len(masas)
        self.Ys = {}
        self.dYsdt = {}
        #self.ts = array([])
        self.fun_derivada = fun_derivada
        self.metodo = metodo
        
        self.t_max = t_max - t_max % timestp
        self.timestp = timestp
        self.cifras_fpos = cifras_fpos
        
        self.periodica = False
        self.periodo = -100.
        
        # Cuadrar vectores Y en diccionario con keys los tiempos
        # #posiciones_0 = np.array(Y_0[: n])
        # #velocidades_0 = np.array(velocidades_0[n :])
        
        ## Agregar info inicial
        self.ts = np.array([0.])
        self.Ys[self.ts[0]] = Y_0
        
        ## Calculo del sistema en cada tiempo hasta self.t_max
        ####TODO Ys antes
        #print 'Ys antes'
        #print self.Ys
        self.calcular_Ys( )
        ####TODO Ys despues
        #print 'Ys despues'
        #print self.Ys
        #print 'ts despues'
        #print self.ts
        
        # Cambiar estado actual
        self.estado_actual = self.cambiar_estado(self.ts[-1])
    
    def aumentar_t_max(self, t_max):
        
        #TODO Check t_max, if less than current, raise Exc, if periodic, raise Exception (to alert)
        
        # Cambiar el tiempo maximo
        self.t_max = self.redondear_t(t_max)
        
        # Agregar Ys adicionales
        self.calcular_Ys(restart = False)
    
    def cambiar_timestep(self, timestp):
        
        #TODO Check timestp
        
        # Cambiar el time step
        self.timestp = timestp
        
        # Recalcular Ys
        self.calcular_Ys(restart = True)
    
    def cambiar_t_max_y_timestep(self, t_max, timestp):
        
        #TODO Check times
        
        # Cambiar timestp y t_max
        self.timestp = timestp
        self.t_max = self.redondear_t(t_max)
        
        # Recalcular Ys
        self.calcular_Ys(restart = True)
    
    def cambiar_estado(self, t):
        
        # Cambiar tiempo y Y actuales
        self.t_actual = t#self.redondear_tiempo(t)
        self.Y_actual = self.Ys[self.t_actual]
        
    def info_actual(self):
        # Retorna el tiempo y el vector Y del estado actual#Careful with copies
        
        return self.t_actual, self.Y_actual
    
    def energia_cinetica(self):
        # Del estado actual
        # Asume que la segunda parte de ls Ys corresponde a las velocidades
        
        vels_act = self.Y_actual[self.N : 2*self.N]
        return np.sum(array([np.sum(vels_act[i]**2) * self.masas[i] / 2 for i in range(self.N)]))
        
    def energia_potencial(self):
        # Del estado actual
        # Asume que la segunda parte de ls Ys corresponde a las velocidades
        e_potencial = 0
        
        for i in range(self.N - 1):
            # Suma de modo que no se repetan elementos
            m = self.masas[i]
            masas_act = self.masas[i + 1: ]
            r = self.Y_actual[i]
            rs_act = self.Y_actual[i + 1 : self.N] - r # Respecto al actual
            
            # Suma de aceleraciones debidas a cada masa
            numeradores = -G * masas_act 
            denominadores = np.sum(rs_act**2, axis = 1)**(1./2)
            
            e_potencial += np.sum(array([numeradores[i] / denominadores[i] for i in range(self.N - i - 1)])) # No igual a división de arrays
        
        return e_potencial
        
    def energia_total(self):
        # Del estado actual
        # Asume que la segunda parte de ls Ys corresponde a las velocidades
        
        return self.energia_cinetica() + self.energia_potencial()
       
    
    def calcular_Ys(self, restart = False):
        # Calcula los Ys hasta el tiempo máximo
        ## Asume que el tiempo máximo es menor al de un periodo, si es que esto tiene sentido
        
        # Si restart, reasigna los tiempos, los Ys, los dYsdt
        ## y empieza en t = 0
        if restart:
            self.ts = array([0.])
            self.Ys = {Ys[self.ts[0]]} #########3
            self.dYsdt = {}
            self.cambiar_estado(self.ts[0])
        
        #TODO Calcular los Ys
        t = self.ts[-1]
        
        while t < self.t_max:
            ####TODO verificar que entra
            ####print 'Entra al while, tiempo ', t
            timestp = self.time_step_adecuado(t)
            # Calcular siguiente Y
            Y = self.metodo(self.fun_derivada, np.append(self.masas, [G], axis = 0), 
                            self.Ys, self.dYsdt, timestep = self.timestp, ts = self.ts)
            
            # If new Y is good enough to determine periodicity, change period stuff, t_max and break
            if prod(vec_iguales_cifras(Y, self.Ys[self.ts[0]], self.cifras_fpos)):
                ####TODO verificar periodicidad
                ####print 'Entra por creer periodicidad, tiempo ', t + timestp
                self.periodica = True
                self.periodo = t
                self.t_max = t
                break
            
            # Advance a timestep
            t += self.timestp
            t = self.redondear_tiempo(t)
            
            # Store new Y and t
            self.Ys[t] = Y
            self.ts = np.append(self.ts, [t], axis = 0)
    
    def time_step_adecuado(self, t): #TODO improve
        
        # Asume t en ts
        if self.timestp != None:
            return self.timestp
        
        return self.timestp
    
    def redondear_tiempo(self, t): #Mejor llamarla en caso de querer utilizar un metodo que no utilice pasos uniformes
        # Asume que t es positivo
        
        if not self.periodica and t > self.t_max + self.timestp: #TODO tal vez cambiar este check a alguien más
            raise Exception('Tiempo muy largo, aumentar primero tiempo maximo')
        
        if self.periodica:
            t = t % self.periodo
            
        #return t - t % self.timestp
        #print 'Redondeando'
        #print 't', t
        #print 'cociente', t/self.timestp
        #print 'round cociente', round(t/self.timestp)
        #print 't, timestep, round cociente', t, self.timestp, round(t/self.timestp, 0)
        return self.timestp * round(t/self.timestp, 0)


In [ ]:
def fun_der_grav(Y, t, constantes, dYsdt):
    
    # Asume Y = array([r1, r2, ..., v1, v2, ...]) con rs y vs array([c1, c2, c3])
    # Asume constantes = [m1, m2, ..., G] con el tipo creo que cualquier sequencia
    
    #TODO Checar longitudes de arrays de masa y Y
    
    # Devolver la derivada si ésta ya ha sido calculada. 
    ##IMPORTANTE:
    ## Asume que no
    ## ocurre recalculo de derivadas, porque asumo que la funcion derivada
    ## solo depende del Y y el t donde se calcula.
    
    if t in dYsdt:
        return dYsdt[t]
    
    masas = array(constantes[: -1])
    G = constantes[-1]
    
    n = len(masas)
    #dYdt = array([]) # Array solucion
    
    # Agregar velocidades a la solucion
    dYdt = Y[n :]
    
    # Agregar aceleraciones
    for i in range(n):
        # Concatena lo que esta a izquierda y derecha para masas y vectores posicion actuales
        masas_act = np.concatenate( (masas[: i], masas[i + 1 :]) , axis = 0) 
        rs_act = np.concatenate( (Y[: i], Y[i + 1 : n]) , axis = 0) - Y[i] # Respecto al actual
        # Suma de aceleraciones debidas a cada masa
        numeradores = G * array([masas_act[i] * rs_act[i] for i in range(n - 1)]) # No igual a multiplicacion de arrays
        denominadores = np.sum(rs_act**2, axis = 1)**(3./2)
        aceleraciones = array([numeradores[i] / denominadores[i] for i in range(n - 1)]) # No igual a división de arrays
        dYdt = np.append(dYdt, [np.sum(aceleraciones, axis = 0)], axis = 0)

    # Agregar resultado al diccionario de derivadas guardadas.
    dYsdt[t] = dYdt #TODO estudiar si el diccionario se pasa completo
    
    return dYdt 